<a href="https://colab.research.google.com/github/adrien50/basicprojectmachinelearning/blob/main/fastapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Machine Learning Model with FastAPI


Building a machine learning model is just one part of the picture. To be of any use in the real world, it must be accessible to users and developers. The easiest and most widely used method for deploying machine learning models is to wrap them inside a REST API. That’s just what we’ll do today, with a trending library — FastAPI.

In [4]:
!pip install colabcode

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 737kB 5.6MB/s 
     |████████████████████████████████| 8.3MB 7.4MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 194kB 44.4MB/s 
     |████████████████████████████████| 430kB 33.6MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 112kB 48.6MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.2-cp36-none-any.whl size=18972 sha256=fe5805c88f5b4019e17545b9dcc1cf15ad918d536bba2ca73fe7f60eda3c2596
  Stored in directory: /root/.cache/pip/wheels/e1/94/52/c88ac157149e3506d377ae531697455b43ab49b76d3f2cc862
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=919d0db37a07896110afa888cb7c784f89019fcd4c2b

In [1]:
!pip install fastapi

     |████████████████████████████████| 51kB 2.3MB/s 
     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 9.2MB 5.4MB/s 


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/kaustubhgupta/Technocolab-Final-Project/master/Data/cleaned.csv",  index_col=None)

In [4]:
df.head()

,track_id,genre_top,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,153,Rock,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018
1,154,Rock,0.970135,0.352946,0.023852,0.957113,0.113261,0.032177,53.758,0.035632
2,155,Rock,0.981657,0.142249,0.912122,0.967294,0.363510,0.087527,91.912,0.034325
3,169,Rock,0.989141,0.225978,0.722835,0.263076,0.092371,0.053406,94.322,0.028347
4,170,Rock,0.886660,0.298518,0.744333,0.920950,0.139587,0.088781,97.880,0.073548


In [5]:
X = df.drop(['track_id', 'genre_top'], axis=1)
y = df.genre_top

In [7]:
import pickle
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

tree = DecisionTreeClassifier(random_state=42)

tree.fit(X_train, y_train)

Pkl_Filename = "model_tree.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(tree, file)

In [8]:
from pydantic import BaseModel

class Music(BaseModel):
    acousticness: float 
    danceability: float 
    energy: float 
    instrumentalness: float 
    liveness: float 
    speechiness: float 
    tempo: float 
    valence: float
    class Config:
        schema_extra = {
            "example": {
                "acousticness": 0.838816, 
                "danceability": 0.542950, 
                "energy": 0.669215,
                "instrumentalness": 0.000006,
                "liveness": 0.105610,
                "speechiness": 0.391221,
                "tempo": 111.894,
                "valence": 0.796073
            }
        }

In [9]:
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_tree.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def get_music_category(data: Music):
    received = data.dict()
    acousticness = received['acousticness']
    danceability = received['danceability']
    energy = received['energy']
    instrumentalness = received['instrumentalness']
    liveness = received['liveness']
    speechiness = received['speechiness']
    tempo = received['tempo']
    valence = received['valence']
    pred_name = model.predict([[acousticness, danceability, energy,
                                instrumentalness, liveness, speechiness, tempo, valence]]).tolist()[0]
    return {'prediction': pred_name}

In [10]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [11]:
server.run_app(app=app)

INFO:     Started server process [199]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://dd7de51bbd96.ngrok.io" -> "http://localhost:10000"
INFO:     154.72.167.97:0 - "GET / HTTP/1.1" 200 OK
INFO:     154.72.167.97:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [199]
